In [ ]:
import numpy as np 
import pandas as pd 


In [ ]:
import sys
import glob
from tqdm import tqdm

In [ ]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import AveragePooling2D
from keras.applications import ResNet50,MobileNetV2,MobileNet,InceptionV3
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input, ConvLSTM2D, BatchNormalization, MaxPooling3D, TimeDistributed
from keras.models import Model
from keras.losses import categorical_crossentropy
from keras.utils import to_categorical
from keras.optimizers import SGD,Adam
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import warnings
import argparse
import pickle
import cv2
import os

In [ ]:
trainX = np.load('/kaggle/input/traintestfile/trainX.npy')
trainY = np.load('/kaggle/input/traintestfile/trainY.npy')
testX = np.load('/kaggle/input/traintestfile/testX.npy')
testY = np.load('/kaggle/input/traintestfile/testY.npy')

## Data Augmentation

In [ ]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [ ]:
valAug = ImageDataGenerator()

In [ ]:
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

# XGBoost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
X_xgb_train = np.array([cv2.resize(img, (64, 64)) for img in trainX])
X_xgb_test = np.array([cv2.resize(img, (64, 64)) for img in testX])
Y_xgb_train = trainY[:, 0]
Y_xgb_test = testY[:, 0]

In [ ]:
X_xgb_train = np.reshape(X_xgb_train, (X_xgb_train.shape[0],X_xgb_train.shape[1]*X_xgb_train.shape[2]*X_xgb_train.shape[3]))
X_xgb_test = np.reshape(X_xgb_test, (X_xgb_test.shape[0],X_xgb_test.shape[1]*X_xgb_test.shape[2]*X_xgb_test.shape[3]))

In [ ]:
model = XGBClassifier(verbosity=0)
model.fit(X_xgb_train, Y_xgb_train, verbose=1, eval_set=[(X_xgb_test, Y_xgb_test)],early_stopping_rounds=5)

In [ ]:
# evaluate the network

y_pred = model.predict(X_xgb_test)
print(classification_report(Y_xgb_test,y_pred, target_names=list(map(str,[0,1]))))

In [ ]:
file_name = "xgb.pkl"

# save
pickle.dump(model, open(file_name, "wb"))

## InceptionV3 model

In [ ]:
baseModel = InceptionV3(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(5, 5))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense((2), activation="softmax")(headModel)

In [ ]:
model = Model(inputs=baseModel.input, outputs=headModel)

In [ ]:
model.trainable = True

## Compile the model

In [ ]:
args = {
    "dataset": "data",
    "model": "model/violence_model.h5",
    "label-bin": "model/lb.pickle",
    "epochs": 10,
    "plot": "plot.png"
    
}


In [ ]:
# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print('-'*100)
print("[INFO] compiling model...")
print('-'*100)
opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / args["epochs"])
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print(model.summary())

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
tf.config.list_physical_devices('GPU')

## Train the model

In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [ ]:
train_generator = trainAug.flow_from_directory("/kaggle/working/train",
                                               batch_size=8,
                                               class_mode="categorical",
                                               seed=42)

# this is a similar generator, for validation data
validation_generator = valAug.flow_from_directory("/kaggle/working/test", batch_size=8, class_mode="categorical",
                                               seed=42)

In [ ]:
H = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n//train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.n//validation_generator.batch_size,
    epochs=args["epochs"])

## Evaluate the network

In [ ]:
# evaluate the network
predictions = model.predict(testX, batch_size=8)
# print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1), target_names=np.array([0,1])))

In [ ]:
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1), target_names=list(map(str,[0,1]))))

In [ ]:
# plot the training loss and accuracy
print('-'*100)
print("[INFO] plot the training loss and accuracy...")
print('-'*100)
N = args["epochs"]
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(args["plot"])

In [ ]:
model.save("inceptionv5.keras")

# Resnet-50

In [15]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [35]:
baseModel = ResNet50(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

In [36]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(5, 5))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense((2), activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)

In [37]:
for layer in baseModel.layers:
    layer.trainable = False

In [38]:
model.compile(loss=categorical_crossentropy,
                  optimizer=Adam(lr=1e-4), metrics=['accuracy'])
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

In [39]:
train_generator = trainAug.flow_from_directory("/kaggle/working/train",
                                               batch_size=16,
                                               class_mode="categorical",
                                               seed=42)

# this is a similar generator, for validation data
validation_generator = valAug.flow_from_directory("/kaggle/working/test", batch_size=16, class_mode="categorical",
                                               seed=42)

Found 15991 images belonging to 2 classes.
Found 3998 images belonging to 2 classes.


In [40]:
print('-'*100)
print("[INFO] training head...")
print('-'*100)
H = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n//train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.n//validation_generator.batch_size,
    epochs=10)

----------------------------------------------------------------------------------------------------
[INFO] training head...
----------------------------------------------------------------------------------------------------
Epoch 1/10
999/999 [==============================] - 286s 284ms/step - loss: 0.5268 - accuracy: 0.7725 - val_loss: 0.2322 - val_accuracy: 0.9039
Epoch 2/10
999/999 [==============================] - 279s 280ms/step - loss: 0.2738 - accuracy: 0.8851 - val_loss: 0.1883 - val_accuracy: 0.9257
Epoch 3/10
999/999 [==============================] - 276s 276ms/step - loss: 0.2408 - accuracy: 0.9008 - val_loss: 0.1752 - val_accuracy: 0.9315
Epoch 4/10
999/999 [==============================] - 275s 275ms/step - loss: 0.2091 - accuracy: 0.9164 - val_loss: 0.1749 - val_accuracy: 0.9300
Epoch 5/10
999/999 [==============================] - 275s 276ms/step - loss: 0.1861 - accuracy: 0.9227 - val_loss: 0.1354 - val_accuracy: 0.9493
Epoch 6/10
999/999 [========================

In [41]:
# evaluate the network
print('-'*100)
print("[INFO] evaluating network...")
print('-'*100)
predictions = model.predict(testX, batch_size=8)
print(classification_report(testY.argmax(axis=1),predictions.argmax(axis=1), target_names=list(map(str,[0,1]))))

----------------------------------------------------------------------------------------------------
[INFO] evaluating network...
----------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1780
           1       0.97      0.97      0.97      2218

    accuracy                           0.96      3998
   macro avg       0.96      0.96      0.96      3998
weighted avg       0.96      0.96      0.96      3998



In [44]:
# plot the training loss and accuracy
print('-'*100)
print("[INFO] plot the training loss and accuracy...")
print('-'*100)
N = 10
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("resnet_plot.png")

----------------------------------------------------------------------------------------------------
[INFO] plot the training loss and accuracy...
----------------------------------------------------------------------------------------------------


In [45]:
model.save("resnet50.keras")

----------------------------------------------------------------------------------------------------
[INFO] serializing network...
----------------------------------------------------------------------------------------------------
